In [13]:
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import numpy as np
import os
import cv2
import time
import pandas as pd

In [2]:
def load_images():
    dir = os.path.join(os.sep,'home','mohit','Desktop','dataset')
    images = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.imread(os.path.join(dir,'Cheque'+str(k),i,i+'_'+j+'.tif'))
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.cvtColor(images['c'+str(k)+'_'+i+'_'+j], cv2.COLOR_BGR2YCrCb)
                except:
                    break
    return images

In [3]:
def n_size(images):
    size = {} #no. of ink pixels .. considering all non white pixels(<th) as ink pixels
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                count = 0
                try:
                    image = images['c'+str(k)+'_'+i+'_'+j][:,:,0]
                    for a in range(0,image.shape[0]):
                        for b in range(0,image.shape[1]):
                            if image[a][b] != 255:
                                count+=1
                    size['c'+str(k)+'_'+i+'_'+j] = count
                   # print('c'+str(k)+'_'+i+'_'+j,count)
                except:
                    #print('..')
                    break  
    return size

In [4]:
def mask(image):
    r = (image==255)
    return r


In [5]:
def calc_lbp(images,dim,sizes,n_points = 8,radius = 1,method = 'uniform'): #‘default’, ‘ror’, ‘uniform’,'nri_uniform'
    hist = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    lbp_pens = local_binary_pattern(images['c'+str(k)+'_'+i+'_'+j][:,:,dim], n_points, radius, method)
                    lbp_pens+= mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*1000
                    
                    if(method=='uniform'):
                        r=11
                    else:
                        r=257
                    
                    (h, _) = np.histogram(lbp_pens.ravel(),range(0,r)) #change range according to encoding
                    s = sizes['c'+str(k)+'_'+i+'_'+j]
                    hist['c'+str(k)+'_'+i+'_'+j] = h/s
                    
                except:
                    break
                   
    return hist

In [6]:
def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

In [7]:
def compute_dist(param1,param2,n1,n2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

In [8]:
def distance_distribution(features,sizes):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' same {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' diff {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                        except:
                            
                            pass
    print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

In [9]:
images = load_images()

In [10]:
sizes = n_size(images)

# using riu2 encoding

In [11]:
hist_Y = calc_lbp(images,0,sizes)
hist_Cb = calc_lbp(images,1,sizes)
hist_Cr = calc_lbp(images,2,sizes)

In [12]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy = dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
26.734756180864732 25.32294401367777 19.525893457372764


In [14]:
Y = pd.DataFrame(hist_Y)
Cb = pd.DataFrame(hist_Cb)
Cr = pd.DataFrame(hist_Cr)

In [16]:
glcm_features_all = pd.concat([Y,Cb,Cr])
glcm_features_all.to_csv('/home/mohit/Desktop/featuresUpdated/LBP_riu2.csv', sep=',')

# using default encoding

In [17]:
hist_Y = calc_lbp(images,0,sizes,method='default')
hist_Cb = calc_lbp(images,1,sizes,method='default')
hist_Cr = calc_lbp(images,2,sizes,method='default')

In [18]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy = dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
20.86354096625282 25.322397315873626 20.90342657885528


In [19]:
Y = pd.DataFrame(hist_Y)
Cb = pd.DataFrame(hist_Cb)
Cr = pd.DataFrame(hist_Cr)

glcm_features_all = pd.concat([Y,Cb,Cr])
glcm_features_all.to_csv('/home/mohit/Desktop/featuresUpdated/LBP_default.csv', sep=',')